# Fall Detection
Machine learning algorithm to detect fall

## 1. Global variables
- Import statements
- Global Defines

In [1]:
import matplotlib as plt
import numpy as np
import os
from csv import reader

from sklearn.model_selection import train_test_split


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils


Using TensorFlow backend.


### Global Defines

In [2]:
# File paths
FILEPATH_MOTION_DATASET = "../logs/motion/"
FILEPATH_STATIONARY_DATASET = "../logs/stationary/"

# Defining the output
STATIONARY = 0
MOTION = 1

# Max value is used for normalization
MAX_ACCEL = 20

# Test Size
TEST_SIZE = 0.30 # 30 percent

# Hyper Parameters
input_dim = 150 #x*50, y*50, z*50
output_dim = nb_classes = 2 # 2 classes
batch_size = 1 
nb_epoch = 20

## 2. Preprocessing

- Load the dataset

1. Extract only the linear accel (last 3 entries)
2. Limit the size to 50 (#25 to #75)
3. Normalize the data (-20 to 20) will get changed to (-1 to 1). All values are clipped at 20
4. Flatten the data - looks like [x1,y1,z1,x2,y2,z2...,x150,y150,z150]

In [3]:
 
# Load a CSV file
def load_csv(filename):
    file = open(filename, "r")
    lines = reader(file)
    dataset = list(lines)
    return np.float_(dataset)

def preprocess_data(data):
    # extract last 3 entries and limit size to 50
    data = data[24:74,3:6]
    data = np.clip(data, -1 * MAX_ACCEL, MAX_ACCEL)
    data = data/MAX_ACCEL
    return data.flatten()

def convert_file_to_data_array(filepath):
    return preprocess_data(load_csv(filepath))

# print("First five samples read from file")
# print(load_csv("../logs/motion/1.txt")[:5])

# print("First five samples preprocessing")
# print(convert_file_to_data_array("../logs/motion/1.txt")[:5])

def load_dataset():
    Y = []

    stationary_files = os.listdir(FILEPATH_STATIONARY_DATASET)
    motion_files = os.listdir(FILEPATH_MOTION_DATASET)
    X = np.zeros(shape=(len(stationary_files) + len(motion_files),input_dim))
    x_index = 0

    for file in stationary_files:
        array = convert_file_to_data_array(FILEPATH_STATIONARY_DATASET+file)
        if array.shape[0] is input_dim:
            X[x_index] = array
            x_index = x_index +1 
            Y.append(STATIONARY)

    for file in os.listdir(FILEPATH_MOTION_DATASET):
        array = convert_file_to_data_array(FILEPATH_MOTION_DATASET+file)
        if array.shape[0] is input_dim:
            X[x_index] = array
            x_index = x_index +1 
            Y.append(MOTION)
    
    #print (x_index)
    return X[:len(Y)],Y

(15, 150)
2250


### Split dataset
### Convert class vectors to binary class matrices

In [4]:
(X, y) = load_dataset()
#print(X)
print (np.array(X).shape)
# print (np.array(X).size)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE)

# print(X_train[:2])
# print(y_train[:2])
# print(y_test)

# print (np.array(X_train).shape)
# print (np.array(X_test).shape)

#X_train = np.array(X_train).reshape(11, input_dim) 
#X_test = np.array(X_test).reshape(5, input_dim) 
# print (np.array(X_train).)
# print (X_test.shape)
# print (y_train.shape)
# print (y_test.shape)

Y_train = np_utils.to_categorical(y_train, nb_classes) 
Y_test = np_utils.to_categorical(y_test, nb_classes)

(15, 150)
2250
(10, 150)
(5, 150)


### Build the model

In [5]:
model = Sequential() 
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 302       
Total params: 302
Trainable params: 302
Non-trainable params: 0
_________________________________________________________________


### Compile the model

In [6]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,verbose=1, validation_data=(X_test, Y_test))

Train on 10 samples, validate on 5 samples
Epoch 1/20
10/10 [==============================] - 0s 10ms/step - loss: 0.7749 - acc: 0.4000 - val_loss: 0.6504 - val_acc: 0.8000
Epoch 2/20
10/10 [==============================] - 0s 1ms/step - loss: 0.7147 - acc: 0.3000 - val_loss: 0.6386 - val_acc: 0.4000
Epoch 3/20
10/10 [==============================] - 0s 836us/step - loss: 0.6752 - acc: 0.4000 - val_loss: 0.6263 - val_acc: 0.4000
Epoch 4/20
10/10 [==============================] - 0s 963us/step - loss: 0.6444 - acc: 0.4000 - val_loss: 0.6177 - val_acc: 0.4000
Epoch 5/20
10/10 [==============================] - 0s 1ms/step - loss: 0.6171 - acc: 0.5000 - val_loss: 0.5967 - val_acc: 0.4000
Epoch 6/20
10/10 [==============================] - 0s 1ms/step - loss: 0.5977 - acc: 0.5000 - val_loss: 0.5805 - val_acc: 0.4000
Epoch 7/20
10/10 [==============================] - 0s 1ms/step - loss: 0.5782 - acc: 0.5000 - val_loss: 0.5673 - val_acc: 0.4000
Epoch 8/20
10/10 [========================

### Display Result

In [7]:
score = model.evaluate(X_test, Y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.3949466645717621
Test accuracy: 1.0
